Imports

In [1]:
from data_translator import TestTranslator, EncodeForGA, GAToScheduleTranslator
from hybrid_solution_data_loader import get_data
from models import Order, SimulationEnvironment
from solver import GASolver
from visualize import visualize_schedule
import objective_function

Read from Source

In [2]:
n_workstations, recipes, operation_times = get_data(0)
recipies, workstations, resources, tasks, _ = TestTranslator().translate(n_workstations, recipes, operation_times)

env = SimulationEnvironment(workstations, tasks, resources, recipies)

Create Orders

In [3]:
earliest_slot = 0
last_slot = 250
recipe_orders = [0, 1, 2, 3, 0, 3, 2, 1, 0, 3] # for dataset 0
orders = []
o_id = 0
for order in recipe_orders:
    orders.append(Order(o_id, 0, 1000, last_slot, [order], 100, 50, False, 0, False, 500)) # for now: use resources to select recipe
    o_id = o_id + 1


Translate for Solver

In [4]:
# translate datamodel to encoding
encoder = EncodeForGA()
values, durations, all_jobs, alternatives = encoder.translate(env, orders) # encoding, duration lookup table, list of all jobs used (probably not needed), possible alternatives for each job
# value encoding for GA: <workstation, start time>, could possibly just be <workstation>
print(values)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Run Solver

In [5]:
# create solver object
solver = GASolver(values, durations, all_jobs, alternatives, env, orders)
solver.initialize(earliest_slot, last_slot, 100, 50)
solver.run()

Done


Result Output

In [6]:
result = solver.get_best()
selected_tasks = solver.jobs # selected task alternatives
fitness = solver.get_best_fitness()
print("Parameters of the best solution : {solution}".format(solution=result))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=abs(fitness)))


Parameters of the best solution : [  0 121   3  98   3 109   1 117   1 109   3 116   1 100   4 118   2  94
   2 106   3 118   3 115   3 116   0 113   0  94   0 111   2 118   4 103
   0 114   0 119   0 116   3 116   1  97   4  98   3 108   3 114   0  94
   1 105   0 108]
Fitness value of the best solution = 527


Translate to Schedule

In [7]:
schedule = GAToScheduleTranslator().translate(result, selected_tasks, env, orders)
visualize_schedule(schedule, env, orders)

Evaluate Schedule

In [8]:
#NOTE: the objective functions in this case assume the solution is feasible, not additional checks
makespan, tardiness, deviation, idle_time, profit = objective_function.calculate_comparison_values(schedule, orders, env)
print(f'Fitness Values:\nMakespane: {makespan}\nTardiness: {tardiness}\nDeviation: {deviation}\nIdle Time: {idle_time}\nProfit: {profit}')

AttributeError: 'NoneType' object has no attribute 'external_id'